In [1]:
# -*- coding: utf-8 -*-
# End-to-end script:
# - Load data, learn DAG with GES, estimate OLS weights W
# - Alpha (random, from OLS residual distribution) and Beta (hp noise) sampling
# - Form W' = Alpha ⊙ W ⊙ (1+Beta)
# - Generate synthetic data via linear SEM with node-specific residual scales
# - Build mulw features with W' and evaluate performance (F_mw, OF_mw)
# - Summaries and CSV outputs

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import networkx as nx

from pathlib import Path
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    f1_score, roc_auc_score, average_precision_score, brier_score_loss
)

from lightgbm import LGBMClassifier
from causallearn.search.ScoreBased.GES import ges
from scipy.stats import pearsonr


# =========================
# Global settings
# =========================
RANDOM_STATE = 42
N_SPLITS = 10

# Original intervention settings (kept for compatibility; not used directly here)
N_ITER = 10
DELTA_RAW = 1.0

TOP_K_EDGES = 25
W_EPS = 1e-12
PRINT_PROGRESS = True
SCENARIO_VAR = "rect_turn"

# Alpha/Beta experiment settings
N_DRAWS = 10           # number of W' samples
BETA_SIGMA = 0.05      # std of Beta ~ N(0, BETA_SIGMA)
GAMMA = 0.25           # alpha strength scaling from residual std (sigma_node)
ALPHA_DIST = "lognormal"  # 'lognormal' or 'normal'


# =========================
# Metrics and modeling
# =========================
def expected_calibration_error(y_true, y_prob, n_bins=10):
    y_true = np.asarray(y_true).astype(int)
    y_prob = np.asarray(y_prob).astype(float)
    if len(np.unique(y_true)) < 2:
        return np.nan
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ece, total = 0.0, len(y_true)
    for b in range(n_bins):
        left, right = bins[b], bins[b+1]
        mask = (y_prob >= left) & (y_prob < right) if b < n_bins-1 else (y_prob >= left) & (y_prob <= right)
        if mask.sum() == 0:
            continue
        acc = y_true[mask].mean()
        conf = y_prob[mask].mean()
        ece += (mask.sum() / total) * abs(acc - conf)
    return float(ece)

def best_threshold_by_f1(y_true, y_prob):
    ts = np.linspace(0.05, 0.95, 19)
    scores = [(t, f1_score(y_true, (y_prob >= t).astype(int), zero_division=0)) for t in ts]
    return float(max(scores, key=lambda x: x[1])[0])

def make_model_lgbm():
    return LGBMClassifier(
        n_estimators=500, learning_rate=0.05, num_leaves=31,
        max_depth=-1, subsample=0.8, colsample_bytree=0.8,
        reg_alpha=0.0, reg_lambda=0.0, random_state=RANDOM_STATE, objective="binary"
    )

def run_cv_lgbm(X, y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag=""):
    if X is None or X.shape[1] == 0:
        return {m: np.nan for m in ["f1","AUPRC","AUROC","Brier","ECE"]}
    X_np, y_np = X.values, y.values.astype(int)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    f1s, auprcs, aurocs, briers, eces = [], [], [], [], []
    for fold, (tr_idx, te_idx) in enumerate(skf.split(X_np, y_np), start=1):
        if PRINT_PROGRESS and tag:
            print(f"[{tag}] fold {fold}/{n_splits}")
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]
        X_tr_sub, X_val, y_tr_sub, y_val = train_test_split(
            X_tr, y_tr, test_size=0.2, random_state=random_state, stratify=y_tr
        )
        clf = make_model_lgbm()
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            clf.fit(X_tr_sub, y_tr_sub)
        y_val_proba = clf.predict_proba(X_val)[:, 1]
        t = best_threshold_by_f1(y_val, y_val_proba)
        y_proba = clf.predict_proba(X_te)[:, 1]
        y_pred = (y_proba >= t).astype(int)
        f1s.append(f1_score(y_te, y_pred, zero_division=0))
        aurocs.append(roc_auc_score(y_te, y_proba))
        auprcs.append(average_precision_score(y_te, y_proba))
        briers.append(brier_score_loss(y_te, y_proba))
        eces.append(expected_calibration_error(y_te, y_proba, n_bins=10))
    return {"f1": np.mean(f1s), "AUPRC": np.mean(auprcs), "AUROC": np.mean(aurocs),
            "Brier": np.mean(briers), "ECE": np.mean(eces)}


# =========================
# DAG learning and OLS weights
# =========================
def learn_dag_and_weights(X_base, cols):
    X_np = X_base.values.astype(float)
    Record = ges(X_np)
    Gmat = Record['G'].graph

    p = len(cols)
    A = np.zeros((p, p), dtype=float)
    for i in range(p):
        for j in range(p):
            if i == j:
                continue
            # j -> i
            if (Gmat[j, i] == 1) and (Gmat[i, j] == -1):
                A[j, i] = 1.0

    Z = StandardScaler().fit_transform(X_np)
    W = np.zeros_like(A, dtype=float)
    for child in range(p):
        parents = np.where(A[child, :] == 1.0)[0]
        if parents.size == 0:
            continue
        y_child = Z[:, child]
        X_par = Z[:, parents]
        coef, *_ = np.linalg.lstsq(X_par, y_child, rcond=None)
        W[child, parents] = coef.astype(float)
    return A, W


# =========================
# Alpha/Beta, SEM, features
# =========================
def compute_residual_scales(Z, A, W):
    p = Z.shape[1]
    G = nx.DiGraph()
    for child in range(p):
        for par in np.where(A[child,:]==1.0)[0]:
            G.add_edge(par, child)
    if not nx.is_directed_acyclic_graph(G):
        raise ValueError("Graph must be DAG")
    topo = list(nx.topological_sort(G))
    sigma = np.zeros(p, dtype=float)
    for node in topo:
        parents = np.where(A[node,:]==1.0)[0]
        if parents.size == 0:
            sigma[node] = 1.0  # standardized Z assumption
            continue
        y_child = Z[:, node]
        y_hat   = Z[:, parents] @ W[node, parents]
        eps = y_child - y_hat
        sigma[node] = float(np.std(eps, ddof=1)) if eps.size>1 else 0.0
    return sigma

def sample_alpha_matrix(A, sigma_node, gamma=GAMMA, dist=ALPHA_DIST, seed=RANDOM_STATE):
    rng = np.random.default_rng(seed)
    p = A.shape[0]
    Alpha = np.ones_like(A, dtype=float)
    for child in range(p):
        parents = np.where(A[child,:]==1.0)[0]
        if parents.size == 0:
            continue
        s = max(1e-6, float(gamma * sigma_node[child]))
        if dist == "lognormal":
            mu, sig = -0.5*(s**2), s  # mean≈1
            Alpha[child, parents] = rng.lognormal(mean=mu, sigma=sig, size=parents.size)
        elif dist == "normal":
            draws = rng.normal(loc=1.0, scale=s, size=parents.size)
            Alpha[child, parents] = np.clip(draws, 1e-3, None)
        else:
            raise ValueError("dist must be 'lognormal' or 'normal'")
    return Alpha

def sample_beta_matrix(A, beta_sigma=BETA_SIGMA, seed=RANDOM_STATE):
    rng = np.random.default_rng(seed)
    Beta = np.zeros_like(A, dtype=float)
    mask = (A==1.0)
    if np.any(mask):
        Beta[mask] = rng.normal(loc=0.0, scale=beta_sigma, size=int(mask.sum()))
    return Beta

def make_W_prime(W, A, Alpha, Beta):
    Wp = np.zeros_like(W, dtype=float)
    mask = (A==1.0)
    Wp[mask] = Alpha[mask] * W[mask] * (1.0 + Beta[mask])
    return Wp

def sem_generate_from_Wprime(Wp, A, n, sigma_node, seed=RANDOM_STATE):
    rng = np.random.default_rng(seed)
    p = A.shape[0]
    G = nx.DiGraph()
    for child in range(p):
        for par in np.where(A[child,:]==1.0)[0]:
            G.add_edge(par, child)
    if not nx.is_directed_acyclic_graph(G):
        raise ValueError("Graph must be DAG")
    topo = list(nx.topological_sort(G))
    Z_new = np.zeros((n, p), dtype=float)
    for node in topo:
        parents = np.where(A[node,:]==1.0)[0]
        if parents.size == 0:
            eps = rng.normal(0.0, 1.0, size=n)  # root noise
            Z_new[:, node] = eps
        else:
            eps = rng.normal(0.0, max(1e-6, sigma_node[node]), size=n)  # node-specific residual scale
            Z_new[:, node] = Z_new[:, parents] @ Wp[node, parents] + eps
    # Standardize columns (to maintain numeric stability; same convention as original Z)
    Z_new = (Z_new - Z_new.mean(axis=0, keepdims=True)) / (Z_new.std(axis=0, ddof=0, keepdims=True) + 1e-12)
    return Z_new

def build_mulw_features_topk(X_df, cols, W_like, A, top_k=TOP_K_EDGES, eps=W_EPS):
    p = A.shape[0]
    flat = []
    for i in range(p):
        for j in range(p):
            if i==j or A[i,j]!=1.0:
                continue
            w = float(W_like[i,j])
            if abs(w)>eps:
                flat.append((i, j, w))
    flat.sort(key=lambda t: abs(t[2]), reverse=True)
    edges = flat[:min(top_k,len(flat))] if top_k is not None else flat
    feats = {}
    for child, parent, w in edges:
        pn, cn = cols[parent], cols[child]
        feats[f"{pn}_mulw_{cn}"] = float(w) * (X_df[pn] * X_df[cn])
    X_feat = pd.DataFrame(feats, index=X_df.index)
    if X_feat.shape[1]==0:
        return X_feat
    var = X_feat.var(axis=0, ddof=0)
    keep = var[~np.isclose(var.values, 0.0)].index
    return X_feat[keep]


# =========================
# Load data and baseline DAG/W
# =========================
df = pd.read_csv("training_data.csv")
if "label" not in df.columns:
    raise ValueError("training_data.csv must contain 'label' column.")

drop_cols = [c for c in ["Unnamed: 0"] if c in df.columns]
feature_cols = [c for c in df.columns if c not in drop_cols + ["label"]]

X_base = df[feature_cols].copy()
y = df["label"].astype(int)
cols = list(feature_cols)

# report dir
REPORT_DIR = Path("reports") / datetime.now().strftime("%Y%m%d_%H%M%S")
REPORT_DIR.mkdir(parents=True, exist_ok=True)

# Learn DAG and OLS W on standardized Z
print("Learning DAG with GES and estimating OLS weights W ...")
A, W = learn_dag_and_weights(X_base, cols)

# Prepare standardized Z and node residual scales from original W
scaler = StandardScaler().fit(X_base.values.astype(float))
Z_orig = scaler.transform(X_base.values.astype(float))
sigma_node = compute_residual_scales(Z_orig, A, W)  # length p

# Baseline O performance on original data
print("Evaluating baseline (O) on original X_base ...")
res_O_orig = run_cv_lgbm(X_base, y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="O-orig")


# =========================
# Alpha/Beta sampling, SEM generation, performance evaluation
# =========================
print("Sampling W', generating SEM data, and evaluating performance ...")
outdir = REPORT_DIR / "alpha_beta_sem"
outdir.mkdir(parents=True, exist_ok=True)

rows = []
scale_factor_summaries = []
Wp_stats_all = []

for k in range(N_DRAWS):
    Alpha = sample_alpha_matrix(A, sigma_node, gamma=GAMMA, dist=ALPHA_DIST, seed=RANDOM_STATE+k)
    Beta  = sample_beta_matrix(A, beta_sigma=BETA_SIGMA, seed=RANDOM_STATE+k)
    Wp    = make_W_prime(W, A, Alpha, Beta)

    # Scale factor stats over edges: alpha*(1+beta)
    mask = (A==1.0)
    if np.any(mask):
        scale_vals = (Alpha[mask] * (1.0 + Beta[mask])).ravel()
        scale_factor_summaries.append({
            "draw": k,
            "n_edges": int(scale_vals.size),
            "scale_mean": float(np.mean(scale_vals)),
            "scale_std": float(np.std(scale_vals, ddof=1)) if scale_vals.size>1 else np.nan,
            "scale_min": float(np.min(scale_vals)),
            "scale_p25": float(np.percentile(scale_vals, 25)),
            "scale_median": float(np.median(scale_vals)),
            "scale_p75": float(np.percentile(scale_vals, 75)),
            "scale_max": float(np.max(scale_vals))
        })
    else:
        scale_factor_summaries.append({
            "draw": k, "n_edges": 0, "scale_mean": np.nan, "scale_std": np.nan,
            "scale_min": np.nan, "scale_p25": np.nan, "scale_median": np.nan, "scale_p75": np.nan, "scale_max": np.nan
        })

    # W vs W' stats (edges only)
    if np.any(mask):
        We, Wpe = W[mask].ravel(), Wp[mask].ravel()
        if We.size>1 and np.std(We)>0 and np.std(Wpe)>0:
            r, _ = pearsonr(We, Wpe)
        else:
            r = np.nan
        Wp_stats_all.append({
            "draw": k,
            "W_mean": float(np.mean(We)) if We.size>0 else np.nan,
            "W_std": float(np.std(We, ddof=1)) if We.size>1 else np.nan,
            "Wp_mean": float(np.mean(Wpe)) if Wpe.size>0 else np.nan,
            "Wp_std": float(np.std(Wpe, ddof=1)) if Wpe.size>1 else np.nan,
            "pearson_r": float(r)
        })
    else:
        Wp_stats_all.append({
            "draw": k,
            "W_mean": np.nan, "W_std": np.nan,
            "Wp_mean": np.nan, "Wp_std": np.nan,
            "pearson_r": np.nan
        })

    # Generate synthetic Z and X using node-specific residual scales (sigma_node)
    Z_new = sem_generate_from_Wprime(Wp, A, n=len(X_base), sigma_node=sigma_node, seed=RANDOM_STATE+k)
    X_new = scaler.inverse_transform(Z_new)
    X_new_df = pd.DataFrame(X_new, columns=cols, index=X_base.index)

    # Build mulw features with W' and evaluate F_mw and OF_mw
    Xf_mw = build_mulw_features_topk(X_new_df, cols, Wp, A, top_k=TOP_K_EDGES, eps=W_EPS)
    if Xf_mw.shape[1] > 0:
        res_F_mw  = run_cv_lgbm(Xf_mw, y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag=f"F_mw-Wp-{k}")
        res_OF_mw = run_cv_lgbm(pd.concat([X_new_df, Xf_mw], axis=1), y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag=f"OF_mw-Wp-{k}")
    else:
        res_F_mw  = {m: np.nan for m in ["f1","AUPRC","AUROC","Brier","ECE"]}
        res_OF_mw = run_cv_lgbm(X_new_df, y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag=f"O-Wp-{k}")

    for met in ["f1","AUPRC","AUROC","Brier","ECE"]:
        rows.append({"draw": k, "bucket": "O_orig", "metric": met, "score": res_O_orig[met]})
        rows.append({"draw": k, "bucket": "F_mw",   "metric": met, "score": res_F_mw[met]})
        rows.append({"draw": k, "bucket": "OF_mw",  "metric": met, "score": res_OF_mw[met]})

# Save performance tables
df_perf = pd.DataFrame(rows)
df_perf.to_csv(outdir/"sem_performance_by_draw_long.csv", index=False, encoding="utf-8-sig")
perf_pivot = df_perf.pivot_table(index=["draw","bucket"], columns="metric", values="score", aggfunc="mean").reset_index()
perf_pivot.to_csv(outdir/"sem_performance_by_draw.csv", index=False, encoding="utf-8-sig")

summary = df_perf.groupby(["bucket","metric"])["score"].agg(["mean","std","count"]).reset_index()
summary.to_csv(outdir/"sem_performance_summary.csv", index=False, encoding="utf-8-sig")

pd.DataFrame(scale_factor_summaries).to_csv(outdir/"scale_factor_summary.csv", index=False, encoding="utf-8-sig")
pd.DataFrame(Wp_stats_all).to_csv(outdir/"W_vs_Wprime_stats.csv", index=False, encoding="utf-8-sig")


# =========================
# Delta features (Δf) summary for best OF_mw draw (by AUROC, then AUPRC, then f1)
# =========================
def topk_edges_by_absW(W_like, A, cols, k=TOP_K_EDGES, eps=W_EPS):
    p = A.shape[0]
    flat = []
    for i in range(p):
        for j in range(p):
            if i==j or A[i,j]!=1.0:
                continue
            w = float(W_like[i,j])
            if abs(w)>eps:
                flat.append((i, j, w, cols[j], cols[i]))  # (child, parent, w, child_name, parent_name)
    flat.sort(key=lambda t: abs(t[2]), reverse=True)
    return flat[:min(k,len(flat))]

def summarize_delta_features(X_df, edges, W_base, A, Alpha, Beta, cols):
    # Δf_ij = (alpha_ij*(1+beta_ij) - 1) * W_ij * (x_i * x_j)
    records = []
    for (child, parent, w, child_name, parent_name) in edges:
        sc = Alpha[child, parent] * (1.0 + Beta[child, parent])
        prod = (X_df[parent_name] * X_df[child_name]).values.astype(float)
        delta_f = (sc - 1.0) * w * prod
        records.append({
            "edge": f"{parent_name}->{child_name}",
            "W": float(w),
            "scale_factor": float(sc),
            "delta_f_mean": float(np.mean(delta_f)),
            "delta_f_std": float(np.std(delta_f, ddof=1)) if delta_f.size>1 else np.nan,
            "delta_f_min": float(np.min(delta_f)),
            "delta_f_p25": float(np.percentile(delta_f,25)),
            "delta_f_median": float(np.median(delta_f)),
            "delta_f_p75": float(np.percentile(delta_f,75)),
            "delta_f_max": float(np.max(delta_f)),
        })
    return pd.DataFrame(records)

# Pick best draw by OF_mw (AUROC, then AUPRC, then f1)
best_tbl = (
    perf_pivot[perf_pivot["bucket"]=="OF_mw"]
    .sort_values(by=["AUROC","AUPRC","f1"], ascending=False)
    .head(1)
)

if best_tbl.shape[0] > 0:
    best_draw = int(best_tbl["draw"].values[0])
    # Re-sample Alpha/Beta with the same seed rule to reproduce W' and data for that draw
    Alpha_star = sample_alpha_matrix(A, sigma_node, gamma=GAMMA, dist=ALPHA_DIST, seed=RANDOM_STATE+best_draw)
    Beta_star  = sample_beta_matrix(A, beta_sigma=BETA_SIGMA, seed=RANDOM_STATE+best_draw)
    Wp_star    = make_W_prime(W, A, Alpha_star, Beta_star)

    # Rebuild X_new for that draw
    Z_new_star = sem_generate_from_Wprime(Wp_star, A, n=len(X_base), sigma_node=sigma_node, seed=RANDOM_STATE+best_draw)
    X_new_star = scaler.inverse_transform(Z_new_star)
    X_new_star_df = pd.DataFrame(X_new_star, columns=cols, index=X_base.index)

    # Delta feature summary on Top-K edges by |W|
    edges_top = topk_edges_by_absW(W, A, cols, k=TOP_K_EDGES, eps=W_EPS)
    df_deltaf = summarize_delta_features(X_new_star_df, edges_top, W, A, Alpha_star, Beta_star, cols)
    df_deltaf.to_csv(outdir/f"delta_features_top{TOP_K_EDGES}_best_draw{best_draw}.csv", index=False, encoding="utf-8-sig")

    # Also save single-draw W vs W' and Alpha stats for reference
    mask = (A==1.0)
    if np.any(mask):
        We, Wpe = W[mask].ravel(), Wp_star[mask].ravel()
        if We.size>1 and np.std(We)>0 and np.std(Wpe)>0:
            r, _ = pearsonr(We, Wpe)
        else:
            r = np.nan
        pd.DataFrame([{
            "draw": best_draw,
            "W_mean": float(np.mean(We)),
            "W_std": float(np.std(We, ddof=1)) if We.size>1 else np.nan,
            "Wp_mean": float(np.mean(Wpe)),
            "Wp_std": float(np.std(Wpe, ddof=1)) if Wpe.size>1 else np.nan,
            "pearson_r": float(r)
        }]).to_csv(outdir/"W_vs_Wprime_single_draw_stats.csv", index=False, encoding="utf-8-sig")

        ae = (Alpha_star[mask]).ravel()
        pd.DataFrame([{
            "draw": best_draw,
            "alpha_n": int(ae.size),
            "alpha_mean": float(np.mean(ae)) if ae.size>0 else np.nan,
            "alpha_std": float(np.std(ae, ddof=1)) if ae.size>1 else np.nan,
            "alpha_min": float(np.min(ae)) if ae.size>0 else np.nan,
            "alpha_p25": float(np.percentile(ae,25)) if ae.size>0 else np.nan,
            "alpha_med": float(np.median(ae)) if ae.size>0 else np.nan,
            "alpha_p75": float(np.percentile(ae,75)) if ae.size>0 else np.nan,
            "alpha_max": float(np.max(ae)) if ae.size>0 else np.nan
        }]).to_csv(outdir/"alpha_single_draw_stats.csv", index=False, encoding="utf-8-sig")

print("REPORT_DIR =", str(REPORT_DIR))
print("Done.")


Learning DAG with GES and estimating OLS weights W ...
Evaluating baseline (O) on original X_base ...
[O-orig] fold 1/10
[LightGBM] [Info] Number of positive: 285, number of negative: 12588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000514 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3030
[LightGBM] [Info] Number of data points in the train set: 12873, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.022139 -> initscore=-3.788010
[LightGBM] [Info] Start training from score -3.788010
[O-orig] fold 2/10
[LightGBM] [Info] Number of positive: 286, number of negative: 12588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000270 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3026
[LightGBM] [Info] Number of data points in the train set: 12874, number of used features: 13
[LightGBM]

In [2]:
# ============================================================
# Feature importance / used feature extraction
# ============================================================

def extract_lgbm_features(X, y, n_splits=N_SPLITS, random_state=RANDOM_STATE):
    """
    Stratified K-Fold를 기반으로 LightGBM 모델이 실제 사용한 feature 목록과
    feature importance를 집계하여 반환합니다.
    """
    X_np, y_np = X.values, y.values.astype(int)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    all_feature_importances = []
    used_features = set()

    for fold, (tr_idx, te_idx) in enumerate(skf.split(X_np, y_np), start=1):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]

        clf = make_model_lgbm()
        clf.fit(X_tr, y_tr)

        # 모델이 실제 사용한 feature 이름
        fold_features = clf.feature_name_
        used_features.update(fold_features)

        # Feature importance 저장
        imp = pd.DataFrame({
            "fold": fold,
            "feature": fold_features,
            "importance": clf.feature_importances_,
        })
        all_feature_importances.append(imp)

    df_imp = pd.concat(all_feature_importances, axis=0)
    df_summary = (
        df_imp.groupby("feature")["importance"]
        .agg(["mean", "std"])
        .sort_values(by="mean", ascending=False)
        .reset_index()
    )

    return list(used_features), df_summary


# 예시: baseline(O) 데이터에서 사용된 feature 확인
used_features, df_importance = extract_lgbm_features(X_base, y)

print(f"모델이 실제로 사용한 feature 개수: {len(used_features)}")
print("상위 10개 중요 feature:")
print(df_importance.head(10))

# 결과 저장 (선택사항)
df_importance.to_csv(REPORT_DIR / "feature_importance_baseline.csv", index=False, encoding="utf-8-sig")
pd.Series(used_features, name="used_features").to_csv(REPORT_DIR / "used_features_baseline.csv", index=False, encoding="utf-8-sig")


[LightGBM] [Info] Number of positive: 356, number of negative: 15736
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000802 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3103
[LightGBM] [Info] Number of data points in the train set: 16092, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.022123 -> initscore=-3.788776
[LightGBM] [Info] Start training from score -3.788776
[LightGBM] [Info] Number of positive: 357, number of negative: 15736
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000581 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3096
[LightGBM] [Info] Number of data points in the train set: 16093, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.022184 -> initscore=-3.785971
[LightGBM] [Info] Start training from score -3.785971
[LightGBM] [Info

In [3]:
# ============================================================
# Extract features (edges) discovered by GES
# ============================================================

def extract_ges_features(A, cols, eps=1e-12):
    """
    GES로 학습한 DAG에서 feature 간 인과 관계(edge) 목록을 추출합니다.
    A: adjacency matrix (child, parent)
    cols: 원본 feature 이름 리스트
    """
    edges = []
    for i in range(A.shape[0]):        # child
        for j in range(A.shape[1]):    # parent
            if i == j:
                continue
            if A[i, j] == 1.0:
                edges.append({
                    "parent_idx": j,
                    "child_idx": i,
                    "parent_name": cols[j],
                    "child_name": cols[i],
                })
    return pd.DataFrame(edges)

# 예시: GES로 얻은 feature 관계 추출
df_ges_edges = extract_ges_features(A, cols)
print(f"GES로 학습된 인과 관계(edge) 개수: {len(df_ges_edges)}")
print(df_ges_edges.head(10))

# CSV로 저장
df_ges_edges.to_csv(REPORT_DIR / "ges_learned_edges.csv", index=False, encoding="utf-8-sig")


GES로 학습된 인과 관계(edge) 개수: 35
   parent_idx  child_idx           parent_name         child_name
0           0          1        leverage_ratio  asset_liabilities
1           2          1                   roe  asset_liabilities
2           4          1            debt_ratio  asset_liabilities
3          12          1             rect_turn  asset_liabilities
4           0          3        leverage_ratio     asset_turnover
5           2          3                   roe     asset_turnover
6           7          3  capitalization_ratio     asset_turnover
7           9          3      totaldebt_invcap     asset_turnover
8          12          3             rect_turn     asset_turnover
9           0          4        leverage_ratio         debt_ratio


In [4]:
# GES + OLS 가중치 포함
def extract_ges_weighted_features(A, W, cols, eps=1e-12):
    records = []
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if i == j or A[i, j] != 1.0:
                continue
            w = float(W[i, j])
            if abs(w) > eps:
                records.append({
                    "parent_name": cols[j],
                    "child_name": cols[i],
                    "weight": w
                })
    return pd.DataFrame(records)

df_ges_weighted = extract_ges_weighted_features(A, W, cols)
df_ges_weighted.to_csv(REPORT_DIR / "ges_weighted_edges.csv", index=False, encoding="utf-8-sig")
print(df_ges_weighted.head(10))


            parent_name         child_name    weight
0        leverage_ratio  asset_liabilities  0.247739
1                   roe  asset_liabilities  0.016587
2            debt_ratio  asset_liabilities -1.051067
3             rect_turn  asset_liabilities  0.121717
4        leverage_ratio     asset_turnover  0.144183
5                   roe     asset_turnover  0.139435
6  capitalization_ratio     asset_turnover -0.059944
7      totaldebt_invcap     asset_turnover  0.131252
8             rect_turn     asset_turnover  0.439656
9        leverage_ratio         debt_ratio  0.707020
